In [2]:
from time import sleep
import re
import pandas as pd
import numpy as np
import requests
import bs4
from bs4 import BeautifulSoup
import urllib
import requests

In [33]:
#"This is Baurs Code.I would usually use this as a reference and work to create my own , but due to the impending due date,
#the time it takes to scrape from the web, and what remains to do in the project. I did create a function to process
#the salary, but I later(too late), realized in order to properly process salaries and adjusted them so calculations 
#are possible from the dataframe the parsing and resetting of salaries had to be done during the scrapper and not in the dataframe. 
#If this is an unacceptable move,please let me know and I will spend the weekend working to create my own."?
def salary(div):
    if div.find("span", {"class":"no-wrap"}) != None:
        js = str(div.find("span", {"class":"no-wrap"}).text.strip()).split()
        #print js
        #print float(re.sub(",", "",re.findall(r"\d+\,\d+|\d+\.\d+",js[0])[0]))
        if js[-1] == 'year':
            if js[1] == '-':
                js1 = float(re.sub(",","",(re.findall(r"\d+\,\d+|\d+\.\d+",js[0])[0])))
                #print js1
                js2 = float(re.sub(",","",(re.findall(r"\d+\,\d+|\d+\.\d+",js[2])[0])))
                #print js2
                js = (js1+js2)/2
                #print js
            else: js = float(re.sub(",","",(re.findall(r"\d+\,\d+|\d+\.\d+",js[0])[0])))
            #print js
        elif js[-1] == 'hour':
            if js[1] == '-':
                #print js[0], js[1], js[2]
                js1 = float(re.findall(r"\d+",js[0])[0])
                #print js1
                js2 = float(re.findall(r"\d+",js[2])[0])
                #print js2
                js = (js1+js2)/2*1600
                #print js
            else: js = float(re.findall(r"\d+",js[0])[0])*1600
        elif js[-1] == 'day':
            if js[1] == '-':
                #print js[0], js[1], js[2]
                js1 = float(re.findall(r"\d+",js[0])[0])
                #print js1
                js2 = float(re.findall(r"\d+",js[2])[0])
                #print js2
                js = (js1+js2)/2*200
                #print js
            else: js = float(re.findall(r"\d+",js[0])[0])*200
        elif js[-1] == 'month':
            if js[1] == '-':
                #print js[0], js[1], js[2]
                js1 = float(re.sub(",","",(re.findall(r"\d+\,\d+|\d+\.\d+",js[0])[0])))
                #print js1
                js2 = float(re.sub(",","",(re.findall(r"\d+\,\d+|\d+\.\d+",js[2])[0])))
                #print js2
                js = (js1+js2)/2*12
                #print js
            else: js = float(re.sub(",","",(re.findall(r"\d+\,\d+|\d+\.\d+",js[0])[0])))*12
    else: js = str('NaN')
    #print js
    return js

In [ ]:
['Washington D.C.', 'New York', 'San Francisco', 'Chicago', 'Austin', 'Jacksonville', 'Indianapolis'\
             'Seattle', 'Los Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Columbus', 'Fort Worth', 'Charlotte' \
             'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'San Antonio', 'San Diego', 'San Jose'\
             'Nashville', 'El Paso', 'Boston', 'Detroit', 'Memphis', 'Oklahoma City', 'Baltimore', 'Las Vegas' \
             'Milwaukee', 'Albuquerque', 'Tucson', 'Fresno', 'Sacramento', 'Mesa', 'Kansas City', 'Raleigh']

In [185]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={cities}&start={0}"
max_results_per_city = 100 # Set at high value of . 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.
columns=['City','Title', 'Company', 'Location','Job Summary', 'Salary']
cities = ['Washington D.C.', 'New York', 'San Francisco', 'Chicago', 'Austin', 'Jacksonville', 'Indianapolis'\
             'Seattle', 'Los Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Columbus', 'Fort Worth', 'Charlotte' \
             'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'San Antonio', 'San Diego', 'San Jose'\
             'Nashville', 'El Paso', 'Boston', 'Detroit', 'Memphis', 'Oklahoma City', 'Baltimore', 'Las Vegas' \
             'Milwaukee', 'Albuquerque', 'Tucson', 'Fresno', 'Sacramento', 'Mesa', 'Kansas City', 'Raleigh']
information = []
for city in cities:
    for start in range(0, max_results_per_city, 10):
        html=requests.get("http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=" + str(city) + "&start=" + str(start))
        sleep(0.5)
        soup=BeautifulSoup(html.text,"html.parser", from_encoding='utf-8')
        for div in soup.find_all(name='div', attrs={'class':'result'}):
            #Indicate row # for index of posting in df
            #row_index = (len(datajobs)+1)
            #create emtpy list to hold all data
            posting = []
            #append city name to list
            posting.append(city)
        #Grab Job title
            try:
                posting.append(div.find('h2', class_='jobtitle').find('a').text)
            except:
                try:
                    title2 = div.find('a', class_='turnstileLink')
                    posting.append(title2.text)
                except:
                    posting.append('No Title')
                #job = title.text
        #Grab Company
            #for b in div.find_all(name='span',attrs={'class':'company'}):
            try:
                posting.append(div.find(name='span',attrs={'class':'company'}).text.strip())
            except:
                posting.append('No Company')
        #Grab Location
            #for c in  div.find_all(name='span',attrs={'class':'location'}):
            try:
                posting.append(div.find(name='span',attrs={'class':'location'}).text)
            except:
                posting.append('No Location')
        #Grab Job Description
            #for d in div.find_all(name='span',attrs={'class':'summary'}):
            try:
                posting.append(div.find(name='span',attrs={'class':'summary'}).text.strip())
            except:
                posting.append('No Description')
        # Grab Salary   
            posting.append(salary(div))
        #appending posting information to df
            information.append(posting)
            pass
information

[['Washington D.C.',
  u'Data Scientist',
  u'Riverside Research',
  u'Washington, DC',
  u'1 year practical work experience with geospatial data or imagery data. Identify data needs and methods for data gathering, extraction, integration, quality, etc...',
  'NaN'],
 ['Washington D.C.',
  u'Data Scientist - Defense & Intelligence',
  u'Elder Research Inc',
  u'Linthicum, MD',
  u'Data Scientist - Defense & Intelligence. We are looking to hire Data Scientists with at least 5 years of technical experience working with data and deploying...',
  'NaN'],
 ['Washington D.C.',
  u'Data Scientist',
  u'Novetta',
  u'Crystal City, VA',
  u'Collaborate with fellow technologists and contribute meaningfully in areas such as data sciences, data security, and architecture design....',
  'NaN'],
 ['Washington D.C.',
  u'Data Scientist_Entry Level',
  u'GS5',
  u'Washington, DC',
  u'The Data Scientist_Entry Level is expected to:. GS5, LLC is looking for a TS/SCI Cleared Data Scientist_Entry Level...

In [187]:
df = pd.DataFrame(information, columns=columns)
df

,City,Title,Company,Location,Job Summary,Salary
0,Washington D.C.,Data Scientist,Riverside Research,"Washington, DC",1 year practical work experience with geospati...,NaN
1,Washington D.C.,Data Scientist - Defense & Intelligence,Elder Research Inc,"Linthicum, MD",Data Scientist - Defense & Intelligence. We ar...,NaN
2,Washington D.C.,Data Scientist,Novetta,"Crystal City, VA",Collaborate with fellow technologists and cont...,NaN
3,Washington D.C.,Data Scientist_Entry Level,GS5,"Washington, DC",The Data Scientist_Entry Level is expected to:...,NaN
4,Washington D.C.,Junior Data Scientist / Visualist,Assured Consulting Solutions,"Washington, DC",We are seeking a highly motivated Junior Data ...,NaN
5,Washington D.C.,"Data Scientist, Junior",Booz Allen Hamilton,"Alexandria, VA","Data Scientist, Junior. Perform as a data scie...",NaN
6,Washington D.C.,Machine Learning Algorithm Developer - US Citi...,Naval Research Lab,"Washington, DC 20375 (AU-Tenleytown area)",Develop machine learning algorithms for a vari...,NaN
7,Washington D.C.,Statistical Analyst,eGlobalTech,"Washington, DC",Volumes of data (e.g. Coordinate with Data Ana...,NaN
8,Washington D.C.,Statistician,Ukpeagvik Iñupiat Corporation/Bowhead Family o...,"Washington, DC",- Provide statistical support for sample desig...,NaN
9,Washington D.C.,Data Scientist/Developer/Engineer,ByteCubed,"Arlington, VA",ByteCubed is seeking a Data Scientist/Develope...,NaN


In [119]:
def eda(dataframe):
    print "Missing Values \n \n", dataframe.isnull().sum(),"\n" #find missing values
    print "Duplicate Rows \n", dataframe.duplicated().sum(),"\n" #find duplicated values
    print "Dataframe Types \n \n", dataframe.dtypes,"\n" #datatypes of each column
    print "Dataframe Shape \n", dataframe.shape,"\n" #number of rows and columns
    print "Dataframe Describe \n \n", dataframe.describe(include='all'),"\n" #Describe all columns
    for feature in dataframe: # Prints unique values for each column 
        print feature
        print dataframe[feature].nunique()

In [188]:
eda(df)

Missing Values 
 
City           0
Title          0
Company        0
Location       0
Job Summary    0
Salary         0
dtype: int64 

Duplicate Rows 
2113 

Dataframe Types 
 
City           object
Title          object
Company        object
Location       object
Job Summary    object
Salary         object
dtype: object 

Dataframe Shape 
(4697, 6) 

Dataframe Describe 
 
            City           Title Company       Location  \
count       4697            4697    4697           4697   
unique        32            1817    1253            587   
top     Portland  Data Scientist  BOEING  United States   
freq         150             300     154            230   

                                              Job Summary Salary  
count                                                4697   4697  
unique                                               2155    115  
top     You'll find yourself collaborating with world-...    NaN  
freq                                                   80   

In [121]:
df[df.duplicated()]

,City,Title,Company,Location,Job Summary,Salary
11,San Antonio,Statistician-Programmer,USAA,"San Antonio, TX",Integrates/transforms disparate data sources a...,NaN


In [189]:
City_Words = Large_df['Salary','Binary','City',[Large_df.loc[:,'academic':'youth']]],columns=['Salary','Binary', 'City',[Large_df.loc[:,'academic':'youth']]])

temp_df =  Large_df[['Salary','Binary','City']]
City_words = pd.concat([temp_df, vocab2], axis=1)
City_words
                           
                           
                           
                           

SyntaxError: invalid syntax (<ipython-input-189-22afabad535c>, line 1)

In [ ]:
## YOUR CODE HERE
X = data['City'] #Feature #Using City over Location as they are both the general area Location of the job
y = data['Binary'] #Aim of Predicting Binary T|F based on City 
#train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.3, random_state=3)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

In [ ]:
y2 = pd.factorize(y)
X = pd.get_dummies(data['City'])

In [ ]:
y2

In [ ]:
X

In [ ]:
#Apply the classifier we trained to the  data
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1, max_depth=5)
rf.fit(X, y)

In [ ]:
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
#Create a random forest classifier.
#clf = RandomForestClassifier(n_jobs=-1)
#Train Classifier to take training features,location, and how it realates to training y, binary
#clf.fit(X, y2)
cv = StratifiedKFold(y, n_folds=10, shuffle=True, random_state=35)
dt = RandomForestClassifier(class_weight='balanced')
s = cross_val_score(dt, X, y, cv=cv, n_jobs=-1)
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forest Tree", s.mean().round(3), s.std().round(3))

In [ ]:
importance = pd.DataFrame(zip(rf.feature_importances_,),
                           index=X.columns,
                           columns=['City Importance']).sort_values('City Importance',
                                                                   ascending=False)
importance.head()

In [ ]:
import matplotlib.pyplot as plt
all(rf.feature_importances_ == np.mean([tree.feature_importances_ for tree in rf.estimators_], axis=0))

importances = rf.feature_importances_
# calculate the standard deviation of feature importances by looping over the trees in the random forest
# 

std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

indices = np.argsort(importances)[::-1]
feature_names = X.columns

# Plot the feature importances of the forest
plt.figure(figsize=(10,10))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), feature_names[indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.show()